In [121]:
import sys
# Add the parent directory to sys.path
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import main
from ast import literal_eval
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [122]:
growth_df = pd.read_csv("data_from_girls/growth_df.csv")
df_skill_role_grouped = pd.read_csv("data_from_girls/hard_skills.csv")
similarity_df = pd.read_csv("data_from_girls/similarity_df.csv")
titles_df = pd.read_csv("data_from_girls/titles_df.csv")
merged_df = pd.read_csv("data_from_girls/merged_df.csv")

In [123]:
merged_df["hourly_wage"] = merged_df["hourly_wage"].str.replace("*", "nan")
merged_df.loc[merged_df["hourly_wage"].str.contains("\*")]
merged_df["Skills"] = merged_df["Skills"].apply(lambda x: literal_eval(x))
merged_df["hourly_wage"] = merged_df["hourly_wage"].astype(float, errors="ignore")
merged_df

,O*NET-SOC Code,Title,Skills,hourly_wage,annual_wage
0,13-2011,Accountants and Auditors,"[Microsoft PowerPoint, accounting, Microsoft E...",43.65,90780
1,15-2011,Actuaries,"[SQL, Microsoft Visual Basic for Applications ...",63.70,132500
2,11-3012,Administrative Services Managers,"[Microsoft PowerPoint, Microsoft Excel, Micros...",58.27,121200
3,25-3011,"Adult Basic Education, Adult Secondary Educati...","[operations research, Microsoft Excel]",31.46,65430
4,41-3011,Advertising Sales Agents,"[Adobe Creative Cloud software, Adobe InDesign...",36.45,75820
...,...,...,...,...,...
399,17-2199,Wind Energy Engineers,"[The MathWorks MATLAB, Microsoft Excel, C++, P...",56.90,118350
400,49-9081,Wind Turbine Service Technicians,"[operations research, SAP software]",31.43,65380
401,43-9022,Word Processors and Typists,"[Microsoft Word, operations research, Microsof...",22.68,47170
402,27-3043,Writers and Authors,"[Microsoft PowerPoint, Adobe Photoshop, TikTok...",42.11,87590


In [124]:
merged_df

,O*NET-SOC Code,Title,Skills,hourly_wage,annual_wage
0,13-2011,Accountants and Auditors,"[Microsoft PowerPoint, accounting, Microsoft E...",43.65,90780
1,15-2011,Actuaries,"[SQL, Microsoft Visual Basic for Applications ...",63.70,132500
2,11-3012,Administrative Services Managers,"[Microsoft PowerPoint, Microsoft Excel, Micros...",58.27,121200
3,25-3011,"Adult Basic Education, Adult Secondary Educati...","[operations research, Microsoft Excel]",31.46,65430
4,41-3011,Advertising Sales Agents,"[Adobe Creative Cloud software, Adobe InDesign...",36.45,75820
...,...,...,...,...,...
399,17-2199,Wind Energy Engineers,"[The MathWorks MATLAB, Microsoft Excel, C++, P...",56.90,118350
400,49-9081,Wind Turbine Service Technicians,"[operations research, SAP software]",31.43,65380
401,43-9022,Word Processors and Typists,"[Microsoft Word, operations research, Microsof...",22.68,47170
402,27-3043,Writers and Authors,"[Microsoft PowerPoint, Adobe Photoshop, TikTok...",42.11,87590


In [99]:
df_skill_role_grouped["Skills"] = df_skill_role_grouped["Skills"].apply(lambda x: literal_eval(x))
df_skill_role_grouped.head(5)

,Title,O*NET-SOC Code,Skills,num_skills
0,Accountants and Auditors,13-2011,"[Intuit QuickBooks, Microsoft Excel, Microsoft...",5
1,Actuaries,15-2011,"[Microsoft Excel, Microsoft Power BI, Microsof...",9
2,Administrative Services Managers,11-3012,"[Microsoft Excel, Microsoft Outlook, Microsoft...",4
3,"Adult Basic Education, Adult Secondary Educati...",25-3011,[Microsoft Excel],1
4,Advertising Sales Agents,41-3011,"[Adobe Creative Cloud software, Adobe Illustra...",11


In [100]:
df_resumes = main.get_resumes("../resumes")
df_resumes = main.resume_extraction(df_resumes)

In [125]:
# Initialize the model
model = SentenceTransformer('all-mpnet-base-v2')
model.max_seq_length = 75
model.tokenizer.padding_side = "right"
model.eval()

def add_eos(input_examples):
    """Helper function to add special tokens between each skill."""
    input_examples = [input_example + model.tokenizer.eos_token for input_example in input_examples]
    return input_examples

In [126]:
# Preprocess the job descriptions' skills
merged_df['Skills_Text'] = merged_df['Skills'].apply(add_eos)
merged_df['Skills_Text'] = merged_df['Skills_Text'].apply(
    lambda x: ' '.join(sorted(set(x))) if isinstance(x, list) else ''
)
job_embeddings = model.encode(
    merged_df['Skills_Text'].tolist(),
    batch_size=32,
    num_workers=4,
    show_progress_bar=False
)
merged_df["skills_embed"] = job_embeddings.tolist()

In [ ]:
def similarity_final_3(cv_df, job_df=merged_df, hard_skills=df_skill_role_grouped, scores_df=similarity_df, titles_df = titles_df, growth_df = growth_df, role=None, wage = 0, parallel=False):
    """
    Calculate cosine similarity between a single CV and multiple job descriptions.
    Optionally filter based on a specific role and include role-specific scores in the output.
    """
    # Initialize the model
    model = SentenceTransformer('all-mpnet-base-v2')
    model.max_seq_length = 75
    model.tokenizer.padding_side = "right"
    model.eval()

    def add_eos(input_examples):
        """Helper function to add special tokens between each skill."""
        input_examples = [input_example + model.tokenizer.eos_token for input_example in input_examples]
        return input_examples

    def missing_skills(skills, cv):
        """Return a list of words from the job description (title and skills) that do not match any keywords in the CV."""
        skill_words = set(skill.lower().strip() for skill in skills if isinstance(skill, str))
        cv_set = set(cv.lower().strip() for cv in cv if isinstance(cv, str))
        missing_words = skill_words - cv_set
        return list(missing_words)

    # Find the generic title (Role) corresponding to the input title (Reported Job Title)
    generic_title = titles_df.loc[titles_df['Reported Job Title'] == role, 'Title'].iloc[0]
    
    # Filter scores_df using the generic title
    filtered_scores_df = scores_df[scores_df['Target_Role'] == generic_title]
    # Map the filtered scores to a dictionary for quick lookup
    role_score_mapping = dict(zip(filtered_scores_df['Role'], filtered_scores_df['composite_tasks_dwas_ksas']))


    # Extract and process the CV's skills
    cv_df['Skills_Text'] = cv_df['Skills'].apply(add_eos)
    cv_df['Skills_Text'] = cv_df['Skills_Text'].apply(
        lambda x: ' '.join(sorted(set(x))) if isinstance(x, list) else ''
    )
    cv_embedding = model.encode(
        cv_df['Skills_Text'].iloc[0],  # Assuming a single CV is provided
        batch_size=1,
        show_progress_bar=False
    )

   job_embeddings = merged_df["skills_embed"]
    # Compute cosine similarity
    similarity_scores = cosine_similarity([cv_embedding], job_embeddings).flatten()
    job_df['similarity_score'] = similarity_scores

    # Normalize similarity scores using z-score
    mean_score = job_df['similarity_score'].mean()
    std_score = job_df['similarity_score'].std()
    if std_score > 0:
        job_df['normalized_similarity_score'] = (job_df['similarity_score'] - mean_score) / std_score
    else:
        job_df['normalized_similarity_score'] = 0  # Assign 0 if std is zero (all scores are identical)

    # Rank the jobs by normalized similarity score
    job_df['rank'] = job_df['normalized_similarity_score'].rank(ascending=False)

    # Add missing skills
    job_df['missing_skills'] = job_df.apply(
        lambda row: missing_skills(
            hard_skills.loc[row.name, 'Skills'],  # Skills from df_skill_role_grouped
            cv_df['Skills'].iloc[0]  # Skills from the first row of df_resumes
        ),
        axis=1
    )

    # Map the growth information using generic title
    role_growth_mapping = dict(zip(growth_df['Occupation'], growth_df['Categories']))
    job_df['role_growth'] = job_df['Title'].map(role_growth_mapping)

    # Fill NaN values in 'role_growth' with 'Not In-Demand'
    job_df['role_growth'].fillna('Not In-Demand', inplace=True)

    # Check for missing salary information
    job_df['annual_wage_variation'] = job_df.apply(
        lambda row: "Info Not Available" if row['annual_wage'] == 0 else row['annual_wage'] - wage,
        axis=1
    )

    job_df_2 = job_df.copy()
    # Add the role-specific scores to a new column
    job_df_2['role_scores'] = job_df_2['Title'].apply(lambda x: role_score_mapping.get(x, None))
    # Rank the jobs by normalized similarity score
    job_df_2['rank'] = job_df_2['role_scores'].rank(ascending=False)

    # Return a DataFrame with the job titles, similarity scores, normalized scores, role-specific scores, and ranks
    ranked_jobs_2 = job_df_2[['rank', 'Title', 'missing_skills', 'annual_wage_variation', 'role_growth']].sort_values(by='rank', ascending=True)
    ranked_jobs = job_df[['rank', 'Title', 'missing_skills', 'annual_wage_variation', 'role_growth']].sort_values(by='rank', ascending=True)
    return ranked_jobs, ranked_jobs_2, job


In [107]:
cosine_similarity_df, role_score_df, job_embeddings  = similarity_final_3(df_resumes, role = 'Brand Manager', wage = 75000)

In [119]:
merged_df["skills_embed"] = job_embeddings.tolist()

In [120]:
merged_df

,O*NET-SOC Code,Title,Skills,hourly_wage,annual_wage,Skills_Text,similarity_score,normalized_similarity_score,rank,missing_skills,role_growth,annual_wage_variation,skills_embed
0,13-2011,Accountants and Auditors,"[Microsoft PowerPoint, accounting, Microsoft E...",43.65,90780,Intuit QuickBooks</s> Microsoft Excel</s> Micr...,0.569187,0.919299,56.0,"[sap software, microsoft excel, microsoft outl...",Rapid Growth; Numerous Job Openings,15780,"[-0.05784626305103302, -0.04213870316743851, -..."
1,15-2011,Actuaries,"[SQL, Microsoft Visual Basic for Applications ...",63.70,132500,Microsoft Excel</s> Microsoft Power BI</s> Mic...,0.541662,0.644427,116.0,"[microsoft excel, sas, r, tableau, microsoft v...",Rapid Growth,57500,"[-0.04978768154978752, -0.02512292191386223, -..."
2,11-3012,Administrative Services Managers,"[Microsoft PowerPoint, Microsoft Excel, Micros...",58.27,121200,Microsoft Excel</s> Microsoft Outlook</s> Micr...,0.540859,0.636404,117.0,"[microsoft word, microsoft powerpoint, microso...",Rapid Growth,46200,"[-0.04334821552038193, -0.034958481788635254, ..."
3,25-3011,"Adult Basic Education, Adult Secondary Educati...","[operations research, Microsoft Excel]",31.46,65430,Microsoft Excel</s> operations research</s>,0.445943,-0.311469,275.0,[microsoft excel],Not In-Demand,-9570,"[-0.028807874768972397, -0.0029662055894732475..."
4,41-3011,Advertising Sales Agents,"[Adobe Creative Cloud software, Adobe InDesign...",36.45,75820,Adobe Creative Cloud software</s> Adobe Illust...,0.485206,0.080631,231.0,"[microsoft excel, salesforce software, microso...",Not In-Demand,820,"[-0.02115662768483162, -0.04073476791381836, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,17-2199,Wind Energy Engineers,"[The MathWorks MATLAB, Microsoft Excel, C++, P...",56.90,118350,C++</s> Microsoft Excel</s> Python</s> The Mat...,0.592558,1.152694,35.0,"[c++, microsoft excel, the mathworks matlab]",Not In-Demand,43350,"[-0.06199513375759125, -0.023795675486326218, ..."
400,49-9081,Wind Turbine Service Technicians,"[operations research, SAP software]",31.43,65380,SAP software</s> operations research</s>,0.575061,0.977956,47.0,[sap software],Rapid Growth,-9620,"[0.009088737890124321, -0.030783526599407196, ..."
401,43-9022,Word Processors and Typists,"[Microsoft Word, operations research, Microsof...",22.68,47170,Microsoft Excel</s> Microsoft Outlook</s> Micr...,0.526596,0.493968,156.0,"[microsoft word, microsoft excel, microsoft ou...",Not In-Demand,-27830,"[-0.03345989063382149, -0.00668953126296401, -..."
402,27-3043,Writers and Authors,"[Microsoft PowerPoint, Adobe Photoshop, TikTok...",42.11,87590,Adobe Photoshop</s> Microsoft Excel</s> Micros...,0.356964,-1.200045,355.0,"[adobe photoshop, microsoft powerpoint, micros...",Not In-Demand,12590,"[-0.032950665801763535, -0.07777083665132523, ..."


In [ ]:
foo, bar = main.role_similarity(df_resumes, merged_df, df_skill_role_grouped, similarity_df, titles_df, growth_df, role = 'Brand Manager', wage = 75000)

In [ ]:
with open("data_from_girls/unique_reported_job_titles.txt", "r") as f:
   thing = f.readlines()
thing

['CEO (Chief Executive Officer)\n',
 'Chief Diversity Officer (CDO)\n',
 'Chief Financial Officer (CFO)\n',
 'Chief Information Officer (CIO)\n',
 'Chief Operating Officer (COO)\n',
 'Chief Technical Officer (CTO)\n',
 'Executive Director\n',
 'Executive Vice President (EVP)\n',
 'Operations Vice President (Operations VP)\n',
 'President\n',
 'Business Manager\n',
 'Center Manager\n',
 'Department Manager\n',
 'District Manager\n',
 'General Manager (GM)\n',
 'Operations Director\n',
 'Operations General Manager (Operations GM)\n',
 'Operations Manager\n',
 'Plant Superintendent\n',
 'Store Manager\n',
 'Account Executive\n',
 'Advertising Manager (Ad Manager)\n',
 'Advertising Sales Manager (Ad Sales Manager)\n',
 'Classified Advertising Manager (Classified Ad Manager)\n',
 'Communications Director\n',
 'Communications Manager\n',
 'Creative Services Director\n',
 'Marketing and Promotions Manager\n',
 'Promotions Director\n',
 'Promotions Manager\n',
 'Account Supervisor\n',
 'Brand 

In [49]:
with open("data_from_girls/unique_reported_job_titles.txt", "r") as f:
   thing = list(f)
thing

['CEO (Chief Executive Officer)\n',
 'Chief Diversity Officer (CDO)\n',
 'Chief Financial Officer (CFO)\n',
 'Chief Information Officer (CIO)\n',
 'Chief Operating Officer (COO)\n',
 'Chief Technical Officer (CTO)\n',
 'Executive Director\n',
 'Executive Vice President (EVP)\n',
 'Operations Vice President (Operations VP)\n',
 'President\n',
 'Business Manager\n',
 'Center Manager\n',
 'Department Manager\n',
 'District Manager\n',
 'General Manager (GM)\n',
 'Operations Director\n',
 'Operations General Manager (Operations GM)\n',
 'Operations Manager\n',
 'Plant Superintendent\n',
 'Store Manager\n',
 'Account Executive\n',
 'Advertising Manager (Ad Manager)\n',
 'Advertising Sales Manager (Ad Sales Manager)\n',
 'Classified Advertising Manager (Classified Ad Manager)\n',
 'Communications Director\n',
 'Communications Manager\n',
 'Creative Services Director\n',
 'Marketing and Promotions Manager\n',
 'Promotions Director\n',
 'Promotions Manager\n',
 'Account Supervisor\n',
 'Brand 

In [55]:
li = list(titles_df["Reported Job Title"].unique())

In [56]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('skills.db')
cursor = conn.cursor()

In [61]:
for el in li:
    print(el)
    cursor.execute('INSERT INTO unique_roles (role) VALUES (?)', (el,))


# Commit changes and close connection
conn.commit()
conn.close()

CEO (Chief Executive Officer)
Chief Diversity Officer (CDO)
Chief Financial Officer (CFO)
Chief Information Officer (CIO)
Chief Operating Officer (COO)
Chief Technical Officer (CTO)
Executive Director
Executive Vice President (EVP)
Operations Vice President (Operations VP)
President
Business Manager
Center Manager
Department Manager
District Manager
General Manager (GM)
Operations Director
Operations General Manager (Operations GM)
Operations Manager
Plant Superintendent
Store Manager
Account Executive
Advertising Manager (Ad Manager)
Advertising Sales Manager (Ad Sales Manager)
Classified Advertising Manager (Classified Ad Manager)
Communications Director
Communications Manager
Creative Services Director
Marketing and Promotions Manager
Promotions Director
Promotions Manager
Account Supervisor
Brand Manager
Business Development Director
Business Development Manager
Commercial Lines Manager
Market Development Executive
Marketing Coordinator
Marketing Director
Marketing Manager
Product 

In [70]:
# Connect to SQLite database
conn = sqlite3.connect('skills.db')
cursor = conn.cursor()
cursor.execute("""
    SELECT role
    FROM unique_roles
    ORDER BY role ASC
""")
roles = list(cursor)

In [74]:
roles[0][0]

'3D Animator (Three-Dimensional Animator)'